REACH SDP implementation from author

In [1]:
import cvxpy as cp

import numpy as np
from scipy.linalg import block_diag

from scipy.special import comb
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import scipy.io
from scipy.io import savemat
from scipy.linalg import block_diag
from scipy import sparse
import torch
import mosek
import time
from NeuralNetwork import NeuralNetwork

`polytope` failed to import `cvxopt.glpk`.
will use `scipy.optimize.linprog`


In [2]:
def get_weights(net,save_model=False,file_name='net'):
    '''
    Export pytorch fully connected network to matlab

    '''

    num_layers = int((len(net)-1)/2)
    dim_in = net[0].weight.shape[1]
    dim_out = net[-1].weight.shape[0]
    hidden_dims = [net[2*i].weight.shape[0] for i in range(0,num_layers)]

    # network dimensions
    dims = [dim_in] + hidden_dims + [dim_out]

    # get weights
    weights = np.zeros((num_layers+1,), dtype=np.object)
    weights[:] = [net[2*i].weight.detach().numpy().astype(np.float64) for i in range(0,num_layers+1)]


    # get biases
    biases = np.zeros((num_layers+1,), dtype=np.object)
    biases[:] = [net[2*i].bias.detach().numpy().astype(np.float64).reshape(-1,1) for i in range(0,num_layers+1)]

    activation = str(net[1])[0:-2].lower()

    
    if save_model:
        # export network data to matlab
        data = {}
        data['net'] = {'weights': weights,'biases':biases, 'dims': dims, 'activation': activation, 'name': file_name}

        scipy.io.savemat(file_name + '.mat', data)

        torch.save(net, file_name + '.pt')
    
    return weights, biases, dims


def IBP_relu(weights,biases,dims,x_min,x_max):

    
    pre_activation_lb = np.empty((0,1))
    pre_activation_ub = np.empty((0,1))
    
    post_activation_lb = np.empty((0,1))
    post_activation_ub = np.empty((0,1))

    
    lb = {}
    ub = {}

    lb[0] = x_min
    ub[0] = x_max

    num_layers = len(dims)-2
    num_neurons = sum(dims[1:-1])
    
    

    for i in range(0,num_layers):

        lb[i+1] = np.maximum(0,weights[i])@lb[i] + np.minimum(0,weights[i])@ub[i] + biases[i]
        pre_activation_lb = np.append(pre_activation_lb,lb[i+1])
        

        ub[i+1] = np.maximum(0,weights[i])@ub[i] + np.minimum(0,weights[i])@lb[i] + biases[i]
        pre_activation_ub = np.append(pre_activation_ub,ub[i+1])
        

        lb[i+1] = np.maximum(lb[i+1],0)
        ub[i+1] = np.maximum(ub[i+1],0)
        
        post_activation_lb = np.append(post_activation_lb,lb[i+1])
        post_activation_ub = np.append(post_activation_ub,ub[i+1])
    
    
        
    return post_activation_lb.reshape(num_neurons,1),post_activation_ub.reshape(num_neurons,1)#pre_activation_lb.reshape(num_neurons,1),pre_activation_ub.reshape(num_neurons,1)#,



def generate_sdp_reach(net,x_min,x_max,c, ADynamics=None, BDynamics=None, CDynamics=None):
    
    
    
    x_min_cp = cp.Parameter(x_min.shape)
    x_max_cp = cp.Parameter(x_max.shape)
    c_cp = cp.Parameter(c.shape)
    
    x_min_cp.value = x_min
    x_max_cp.value = x_max
    c_cp.value = c

    
    
    weights, biases, dims = get_weights(net,save_model=False,file_name='net')
    
    dim_in = dims[0]
    dim_out = dims[-1] if ADynamics is None else ADynamics.shape[0]
    num_neurons = sum(dims[1:-1])
    dim_last_hidden = dims[-2]

    
    b_cp = cp.Variable((1,1))
    tau = cp.Variable((dim_in,1),nonneg=True)
    P_in = cp.bmat([[-2*cp.diag(tau),cp.diag(tau)@(x_min+x_max)],[(x_min.T+x_max.T)@cp.diag(tau), -2.0*x_min.T@cp.diag(tau)*x_max]])

    CM_in = np.block([[np.eye(dim_in),np.zeros((dim_in,num_neurons+1))],[np.zeros((1,dim_in+num_neurons)),1]])

    M_in = CM_in.T@P_in@CM_in


    A = block_diag(*weights[0:-1])
    A = np.block([A,np.zeros((A.shape[0],dim_last_hidden))])
    bb = np.concatenate(biases[0:-1],axis=0)
    B = np.block([np.zeros((num_neurons,dim_in)),np.eye(num_neurons)])

    CM_mid = np.block([[A,bb],[B,np.zeros((B.shape[0],1))],[np.zeros((1,B.shape[1])),1]])


    nu = cp.Variable((num_neurons,1),nonneg=True)
    eta = cp.Variable((num_neurons,1),nonneg=True)
    lam = cp.Variable((num_neurons,1))
    mu = cp.Variable((num_neurons,1),nonneg=True)


    lb,ub = IBP_relu(weights,biases,dims,x_min,x_max)
    
    

    Q11 = np.zeros((num_neurons,num_neurons))
    Q12 = cp.diag(lam)
    Q13 = -nu
    Q22 = -2*cp.diag(lam)-2*cp.diag(mu)
    Q23 = eta+nu+cp.diag(mu)@(lb+ub)
    Q33 = np.zeros((1,1))-2*lb.T@cp.diag(mu)*ub

    Q_mid = cp.bmat([[Q11, Q12, Q13],[Q12.T, Q22, Q23],[Q13.T, Q23.T, Q33]])

    M_mid = CM_mid.T@Q_mid@CM_mid
    if ADynamics is None:
        CM_out = np.block([[np.zeros((dim_out,dim_in+num_neurons-dim_last_hidden)), weights[-1], biases[-1]],
                           [np.zeros((1,dim_in+num_neurons)), 1]])
    else:
        if CDynamics is None:
            CDynamics = np.zeros_like(BDynamics @ biases[-1])


        CM_out = np.block([[ADynamics, np.zeros((ADynamics.shape[0], dim_in + num_neurons - ADynamics.shape[1] - weights[-1].shape[1])),
                            BDynamics @ weights[-1], BDynamics @ biases[-1] + CDynamics],
                           [np.zeros((1,dim_in+num_neurons)), 1]])

    S = cp.bmat([[np.zeros((dim_out,dim_out)), c_cp],[c_cp.T, -2*b_cp]])

    M_out = CM_out.T@S@CM_out 

    const = [M_in+M_mid+M_out<<0]

    obj = b_cp

    prob = cp.Problem(cp.Minimize(obj),const)

    
    
    return prob


    
    


In [4]:
fileName = "doubleIntegratorCorrect"
pathToStateDictionary = "Networks/" + fileName +  ".pth"
network = NeuralNetwork(pathToStateDictionary)
net = network.Linear
horizon = 5
A = None
B = None
C = None
if "doubleIntegrator" in pathToStateDictionary:
    A = np.array([[1., 1], [0, 1]])

    B = np.array([[0.5], [1]])
    C = np.array([[0.], [0.]])

    # lowerCoordinate = torch.Tensor([1., 1.5]).to(device)
    # upperCoordinate = torch.Tensor([2., 2.5]).to(device)

    x_min = np.array([[2.5], [-0.25]])
    x_max = np.array([[3.], [0.25]])

In [5]:
weights, biases, dims = get_weights(net,save_model=True,file_name='net')


cs = np.vstack([np.eye(2), -np.eye(2)])
dataDictionary = {}
for j in range(horizon):
    drawingBounds = []
    for i in range(4):
        c = cs[i:i + 1, :].T
        sdp = generate_sdp_reach(net,x_min,x_max,c, ADynamics=A, BDynamics=B, CDynamics=C)
        sdp.solve(solver=cp.MOSEK,verbose=False,warm_start=True)
        drawingBounds.append([sdp.objective.value])
    dataDictionary[j + 1] = {"A": cs, "d": np.array(drawingBounds)}
    x_max[0] = drawingBounds[0][0]
    x_max[1] = drawingBounds[1][0]
    x_min[0] = -drawingBounds[2][0]
    x_min[1] = -drawingBounds[3][0]
torch.save(dataDictionary, "Output/drawing" + fileName + ".pth")

/tmp/ipykernel_651368/1730023611.py:16: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  weights = np.zeros((num_layers+1,), dtype=np.object)
/tmp/ipykernel_651368/1730023611.py:21: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  biases = np.zeros((num_layers+1,), dtype=np.object)
/home/taha/.local/lib/python3.8/site-packages/cvxpy/expressions/expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been 

In [3]:
ts = time.time()
sdp.solve(solver=cp.SCS,verbose=True,warm_start=True)
print(time.time()-ts,sdp.objective.value)

NameError: name 'sdp' is not defined

In [7]:
sdp.objective.value

1.5679090262263344

In [5]:
ts = time.time()
sdp.solve(solver=cp.CVXOPT,verbose=True)
print(time.time()-ts,sdp.objective.value)

     pcost       dcost       gap    pres   dres   k/t
 0: -2.3555e-01 -7.5878e-01  1e+03  1e+01  7e+01  1e+00
 1:  2.9903e-01 -4.4780e-01  3e+02  7e+00  3e+01  5e-03
 2: -5.1050e-02 -1.8510e-01  2e+01  1e+00  7e+00  1e-02
 3:  3.8142e-01  2.4941e-01  2e+01  1e+00  7e+00  1e-02
 4:  3.4025e-01  2.1624e-01  2e+01  1e+00  6e+00  2e-02
 5:  4.8676e-01  3.7610e-01  2e+01  1e+00  6e+00  2e-02
 6:  7.7571e-01  7.1223e-01  1e+01  7e-01  3e+00  1e-02
 7:  7.5093e-01  6.8860e-01  1e+01  7e-01  3e+00  1e-02
 8:  8.4471e-01  7.8850e-01  1e+01  6e-01  3e+00  1e-02
 9:  1.0322e+00  1.0038e+00  6e+00  3e-01  2e+00  6e-03
10:  1.0753e+00  1.0474e+00  6e+00  3e-01  2e+00  6e-03
11:  1.3199e+00  1.3035e+00  4e+00  2e-01  9e-01  4e-03
12:  1.3621e+00  1.3466e+00  4e+00  2e-01  9e-01  4e-03
13:  1.7527e+00  1.7463e+00  2e+00  8e-02  4e-01  2e-03
14:  1.7631e+00  1.7567e+00  2e+00  8e-02  4e-01  2e-03
15:  1.9217e+00  1.9175e+00  1e+00  5e-02  2e-01  1e-03
16:  2.1294e+00  2.1278e+00  5e-01  2e-02  9e-02  

In [8]:
ts = time.time()
sdp.solve(solver=cp.MOSEK,verbose=True,warm_start=True)
print(time.time()-ts,sdp.objective.value)

                                     CVXPY                                     
                                     v1.2.0                                    
(CVXPY) Sep 12 12:36:59 PM: Your problem has 801 variables, 1 constraints, and 10 parameters.
(CVXPY) Sep 12 12:36:59 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Sep 12 12:36:59 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Sep 12 12:36:59 PM: Compiling problem (target solver=MOSEK).
(CVXPY) Sep 12 12:36:59 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> MOSEK
(CVXPY) Sep 12 12:36:59 PM: Applying reduction Dcp2Cone
(CVXPY) Sep 12 12:36:59 PM: Applying reduction CvxAttr2Constr
(CVX

In [9]:
sdp.objective.value

1.5680409906206327